In [1]:
import syft
import syft.nn as nn
from syft.controller import tensors, models
import imp
imp.reload(syft.controller)
imp.reload(syft.nn)
imp.reload(syft)

import numpy as np
from syft import FloatTensor
import torch
from torch.autograd import Variable

In [2]:
input = FloatTensor([[0,0,1],[0,1.0,1],[1,0,1],[1,1,1]], autograd=True)
target = FloatTensor([[0],[0],[1],[1]]).autograd(True)
grad = FloatTensor([[1],[1],[1],[1]]).autograd(False)

np.random.seed(1)

weights1 = FloatTensor(np.random.rand(3,4)).autograd(True)
weights2 = FloatTensor(np.random.rand(4,1)).autograd(True)

input_t = Variable(torch.FloatTensor(input.to_numpy()), requires_grad=True)
target_t = Variable(torch.FloatTensor(target.to_numpy()), requires_grad=True)
weights1_t = Variable(torch.FloatTensor(weights1.to_numpy()), requires_grad=True)
weights2_t = Variable(torch.FloatTensor(weights2.to_numpy()), requires_grad=True)

grad_t = Variable(torch.FloatTensor(grad.to_numpy()))

In [8]:
weights1_t.grad = None
weights2_t.grad = None
target_t.grad = None
input_t.grad = None

layer_1_t = input_t.mm(weights1_t).sigmoid()
layer_2_t = layer_1_t.mm(weights2_t).sigmoid()

diff_t = layer_2_t - target_t
loss_t = diff_t ** 2
loss_t.backward(grad_t)

#print(loss_t.sum().data[0])

weights1_t.data -= weights1_t.grad.data
weights2_t.data -= weights2_t.grad.data
weights1_t.grad.data * 100


 0.2735 -3.6114  1.7777 -2.3399
-0.1450  2.5557 -0.7018  1.2044
-0.2877  5.0704 -1.4171  2.4817
[torch.FloatTensor of size 3x4]

In [9]:
weights2_t.grad.data


 0.1269
 0.1091
 0.1616
 0.1567
[torch.FloatTensor of size 4x1]

In [10]:
layer_1 = input.mm(weights1).sigmoid()
layer_2 = layer_1.mm(weights2).sigmoid()

diff = (layer_2 - target)
loss = diff ** 2 # Mean Squared Error Loss

loss.backward(grad)
weights1 -= weights1.grad()
weights2 -= weights2.grad()

In [11]:
weights1.grad() * 100

[[ 0.2734909 -3.611425   1.777665  -2.339926 ]
 [-0.1450036  2.55574   -0.7018257  1.204421 ]
 [-0.2877049  5.070414  -1.417091   2.481702 ]]
[syft.FloatTensor:38 grad:None size:3x4 c:[] p:[35, 39] init:mul_scalar]

	-----------creators-----------
	[syft.FloatTensor:35 grad:None size:3x4 c:[38] p:[33, 31] init:mm]
	[syft.FloatTensor:39 grad:None size:1 c:[] p:[] init:mul_scalar]
	------------------------------



In [12]:
weights2.grad()

[[ 0.1269271]
 [ 0.109072 ]
 [ 0.1615881]
 [ 0.15669  ]]
[syft.FloatTensor:36 grad:None size:4x1 c:[] p:[24, 22] init:mm]

	-----------creators-----------
	[syft.FloatTensor:24 grad:None size:4x4 c:[36] p:[7] init:transpose]
	[syft.FloatTensor:22 grad:None size:4x1 c:[25, 36] p:[21, 16] init:mul_elem]
	------------------------------



In [13]:
tensors[7]

[[ 0.5928485  0.6099998  0.6026495  0.6512309]
 [ 0.6253144  0.6211784  0.6459926  0.7194341]
 [ 0.6896269  0.7662356  0.6028802  0.719698 ]
 [ 0.7180458  0.7745909  0.646213   0.7790547]]
[syft.FloatTensor:7 grad:24 size:4x4 c:[8, 25, 35] p:[6] init:sigmoid]

	-----------children-----------
	[syft.FloatTensor:8 grad:22 size:4x1 c:[9] p:[7, 5] init:mm]
	[syft.FloatTensor:25 grad:None size:4x4 c:[26, 29] p:[7] init:copy]
	[syft.FloatTensor:35 grad:None size:4x4 c:[36] p:[7] init:transpose]
	------------------------------

	-----------creators-----------
	[syft.FloatTensor:6 grad:30 size:4x4 c:[7] p:[1, 4] init:mm]
	------------------------------



In [14]:
tensors[25].autograd()

False

In [15]:
tensors[35].autograd()

True